In [39]:
import pandas as pd
import sys, os
import inflect
from tqdm import tqdm
import requests

# Add the parent directory of this notebook to the Python path
sys.path.append(os.path.abspath('..'))

import score
import novel
import search_set
import indicator
import literature


# Surpress NLP Mask Warning for Apple Silicon
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*encoder_attention_mask.*BertSdpaSelfAttention\.forward"
)

## Example Code for Identifying Literature to Curate
Demo notebook demonstrating how to use NLP methods to identify literature containing possible interactions for prioritization, manual review, and inclusion as supporting evidence in DGIdb.

In [ ]:
gene = 'BCL2'
search_set.generate_search_set(gene)

In [2]:
abstracts = pd.read_csv('abstracts.csv')
abstracts

,Title,Abstract
0,39796006,"In the tissue regeneration field, stem cell tr..."
1,40524014,"Follicular lymphoma (FL), marginal zone lympho..."
2,40393040,Primary cutaneous diffuse large B cell lymphom...
3,40619042,"Primary ovarian insufficiency (POI), which aff..."
4,40444042,This investigation sought to explore the inhib...
...,...,...
39523,25873999,The role played by microRNAs in the deregulati...
39524,34394999,Late-onset posttransplant lymphoproliferative ...
39525,22431999,Several studies demonstrated that treatment wi...
39526,31465999,Antiviral interferons (IFN-alpha/beta) are pos...


In [11]:
import importlib
importlib.reload(novel)
results = novel.batch(abstracts['Abstract'])
results

Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
100%|██████████| 39528/39528 [1:27:22<00:00,  7.54it/s]


,original_text,entity_group,score,word,start,end
0,"Follicular lymphoma (FL), marginal zone lympho...",GENETIC,0.999927,CD47 - blocking antibody,273,295
1,"Follicular lymphoma (FL), marginal zone lympho...",GENETIC,0.999985,CD47,306,310
2,"Follicular lymphoma (FL), marginal zone lympho...",GENETIC,0.999992,SIRPα,311,316
3,"Follicular lymphoma (FL), marginal zone lympho...",GENETIC,0.999992,anti - CD20 antibody,403,421
4,"Follicular lymphoma (FL), marginal zone lympho...",GENETIC,0.882086,rituximab,422,431
...,...,...,...,...,...,...
712789,This study aimed to determine the effect of 8-...,DISEASE,0.999993,T2D,596,599
712790,This study aimed to determine the effect of 8-...,DISEASE,0.937653,insulin,781,788
712791,This study aimed to determine the effect of 8-...,DISEASE,0.999973,musculoaponeurotic fibrosarcoma,826,857
712792,This study aimed to determine the effect of 8-...,DISEASE,0.999990,T2D,1078,1081


In [15]:
results.to_excel('checkpoint.xlsx')

In [24]:
def _singularize(word):
    inflector = inflect.engine()
    return inflector.singular_noun(word) or word

def _normalize_therapy(word):
    try:
        r = requests.get(
            f'https://normalize.cancervariants.org/therapy/normalize?q={word}&infer_namespace=true',
            timeout=10  # Set timeout for network reliability
        )
        r.raise_for_status()
        response = r.json()

        if isinstance(response, dict) and response.get('match_type') is not None:
            if response['match_type'] != 0:
                return [
                    response['match_type'],
                    response['therapy']['id'],
                    response['therapy']['name']
                ]
            else:
                return [0, None, None]  # Not matched
        else:
            return ['Unexpected Response Format', None, None]
    except requests.exceptions.RequestException as e:
        return ['HTTP Error', str(e), None]
    except Exception as e:
        return ['Failure to Normalize', str(e), None]

# Main loop
checkpoint_interval = 5000
output_base = "normalized_results_checkpoint"
for idx, (index, row) in enumerate(tqdm(results[results['entity_group'] == 'CHEMICAL'].iterrows()), 1):
    word = _singularize(row['word'])
    norm_result = _normalize_therapy(word)

    results.at[index, 'concept_match_type'] = norm_result[0]
    results.at[index, 'concept_id'] = norm_result[1]
    results.at[index, 'concept_label'] = norm_result[2]

    if idx % checkpoint_interval == 0:
        checkpoint_filename = f"{output_base}_checkpoint_{idx}.xlsx"
        results.to_excel(checkpoint_filename, index=False)
        print(f"Checkpoint saved at row {idx} -> {checkpoint_filename}")

# Final save after loop completes
final_filename = f"{output_base}_final.xlsx"
results.to_excel(final_filename, index=False)
print(f"Final results saved -> {final_filename}")

1507it [06:03,  3.92it/s]/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_20542/1454545004.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Failure to Normalize' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results.at[index, 'concept_match_type'] = norm_result[0]
5000it [20:29, 18.59s/it]

Checkpoint saved at row 5000 -> normalized_results_checkpoint_checkpoint_5000.xlsx


10000it [40:53, 20.07s/it]

Checkpoint saved at row 10000 -> normalized_results_checkpoint_checkpoint_10000.xlsx


15000it [59:47, 19.45s/it]

...

Checkpoint saved at row 90000 -> normalized_results_checkpoint_checkpoint_90000.xlsx


95001it [4:27:55, 11.33s/it]

Checkpoint saved at row 95000 -> normalized_results_checkpoint_checkpoint_95000.xlsx


96459it [4:31:13,  5.93it/s]


Final results saved -> normalized_results_checkpoint_final.xlsx


In [ ]:
tdf = results[(results['concept_match_type']!=0) & (results['concept_match_type'].isna()==False)].reset_index(drop=True)

condensed_results = tdf.groupby('original_text').apply(
    lambda group: pd.Series({
        'DRUG_LABELS': ' | '.join(group.loc[group['entity_group'] == 'CHEMICAL', 'concept_label'].dropna().astype(str).unique()),
        'DRUG_IDS': ' | '.join(group.loc[group['entity_group'] == 'CHEMICAL', 'concept_id'].dropna().astype(str).unique())
    })
).reset_index()

condensed_results

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_20542/2943807529.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  condensed_results = tdf.groupby('original_text').apply(


,original_text,DRUG_LABELS,DRUG_IDS
0,ADLE was administered to high-fat diet treate...,nitric oxide | palmitate | oxygen | nitrite io...,normalize.therapy.rxcui:7442 | normalize.thera...
1,GOT1-bearing BALB/c nude mice were treated wi...,sonidegib,normalize.therapy.rxcui:1659191
2,"In this study, we have investigated the cytot...",lactate | ACRIDINE ORANGE | PROPIDIUM | oxygen,normalize.therapy.rxcui:114202 | normalize.the...
3,PNS were identified from the Traditional Chin...,"glutathione | malondialdehyde | ESTROGENS, CON...",normalize.therapy.rxcui:4890 | normalize.thera...
4,Several databases were screened for bioactive...,Tenamfetamine | glutathione,normalize.therapy.ncit:C80152 | normalize.ther...
...,...,...,...
15320,β-thalassemias are common hemoglobinopathies d...,oxygen,normalize.therapy.rxcui:7806
15321,δ-Tocotrienol is a naturally occurring proteas...,CERAMIDE,normalize.therapy.chembl:CHEMBL155886
15322,The malondialdehyde (MDA) level and TA count ...,malondialdehyde | Tenamfetamine | tantalum,normalize.therapy.rxcui:1657018 | normalize.th...
15323,• To investigate the role that oxidative stres...,streptozocin | glutathione | oxygen,normalize.therapy.rxcui:10114 | normalize.ther...


In [43]:
merged_df = pd.merge(
    abstracts,
    condensed_results,
    left_on='Abstract',
    right_on='original_text',
    how='left'
)
merged_df = merged_df[merged_df['DRUG_LABELS'].isnull()==False].reset_index(drop=True)
merged_df

,Title,Abstract,original_text,DRUG_LABELS,DRUG_IDS
0,40393040,Primary cutaneous diffuse large B cell lymphom...,Primary cutaneous diffuse large B cell lymphom...,cyclophosphamide anhydrous | doxorubicin hydro...,normalize.therapy.rxcui:1545988 | normalize.th...
1,40619042,"Primary ovarian insufficiency (POI), which aff...","Primary ovarian insufficiency (POI), which aff...",estradiol valerate | progesterone | Luteolin |...,normalize.therapy.rxcui:24395 | normalize.ther...
2,40444042,This investigation sought to explore the inhib...,This investigation sought to explore the inhib...,wogonin | bromocriptine,normalize.therapy.rxcui:2001619 | normalize.th...
3,40447053,The fate of granulosa cells determines ovarian...,The fate of granulosa cells determines ovarian...,acetate | propionate | butyric acid,normalize.therapy.rxcui:164 | normalize.therap...
4,40512124,BCL-2 is an anti-apoptotic protein expressed b...,BCL-2 is an anti-apoptotic protein expressed b...,venetoclax,normalize.therapy.rxcui:1747556
...,...,...,...,...,...
15399,36042999,Background The Oncotype DX Recurrence Score (O...,Background The Oncotype DX Recurrence Score (O...,"ESTROGENS, CONJUGATED",normalize.therapy.chembl:CHEMBL1201649
15400,38580999,Smilax china L. (SCL) is a traditional herbal ...,Smilax china L. (SCL) is a traditional herbal ...,ASTILBIN,normalize.therapy.chembl:CHEMBL486017
15401,34376999,Glioblastoma multiforme (GBM) is the primary a...,Glioblastoma multiforme (GBM) is the primary a...,curcumin | BISDEMETHOXYCURCUMIN | DEMETHOXYCUR...,normalize.therapy.rxcui:2955 | normalize.thera...
15402,30515999,Lung squamous cell carcinoma (SCC) accounts fo...,Lung squamous cell carcinoma (SCC) accounts fo...,honokiol,normalize.therapy.rxcui:2562546


### Generate Score

In [44]:
dgidb_df = pd.read_csv('search/2025-08-13_BCL2_clin_score.csv')
dgidb_df = dgidb_df.drop_duplicates(subset=['Drug','Gene'], keep='first')
dgidb_df.head()

,nomenclature,Gene,long_name,gene_concept_id,interaction_score,drug_specificity,gene_specificity,evidence_score,source_db_name,source_db_version,...,directionality,definition,reference,drug_name,nomenclature-2,Drug,approved,immunotherapy,anti_neoplastic,drug_concept_id
0,Gene Symbol,BCL2,BCL2 apoptosis regulator,hgnc:990,2.284732,4.141413,0.183893,3.0,TALC,12-May-16,...,1.0,"In inhibitor interactions, the drug binds to a...","<a href=""https://en.wikipedia.org/wiki/Enzyme_...",OBLIMERSEN,Primary Name,OBLIMERSEN,False,False,True,ncit:C1870
3,Gene Name,BCL2,BCL2 apoptosis regulator,hgnc:990,1.523154,4.141413,0.183893,2.0,NCI,14-Sep-17,...,NaN,NaN,NaN,DOLASTATIN 10,Primary Name,DOLASTATIN 10,False,False,False,ncit:C1300
4,Gene Name,BCL2,BCL2 apoptosis regulator,hgnc:990,1.523154,4.141413,0.183893,2.0,NCI,14-Sep-17,...,NaN,NaN,NaN,EPISIALIN,Primary Name,MUC-1 ANTIGEN,False,False,False,ncit:C2407
5,Gene Name,BCL2,BCL2 apoptosis regulator,hgnc:990,1.523154,4.141413,0.183893,2.0,NCI,14-Sep-17,...,NaN,NaN,NaN,MICELLAR PACLITAXEL,Primary Name,MICELLAR PACLITAXEL,False,False,False,ncit:C29256
6,Gene Name,BCL2,BCL2 apoptosis regulator,hgnc:990,1.523154,4.141413,0.183893,2.0,NCI,14-Sep-17,...,NaN,NaN,NaN,BEAUVERICIN,Primary Name,BEAUVERICIN,False,False,False,ncit:C1011


In [74]:
import importlib
importlib.reload(indicator)

indicator.generate_interaction_evidence(merged_df, dgidb_df)

Results saved to 2025-08-19_BCL2.zip!
